#### Queries - Exploración de Tablas DuckLake
##### Configuración
> Asegúrate de tener un archivo `.env` en esta carpeta con las credenciales necesarias.


In [20]:
# Instalar dependencias si es necesario
# !pip install duckdb python-dotenv

In [21]:
import os
import duckdb
from dotenv import load_dotenv

# Cargar variables de entorno desde .env
load_dotenv(dotenv_path='.env', override=True)

# Configuración desde variables de entorno
POSTGRES_HOST = os.getenv('POSTGRES_HOST')
POSTGRES_PORT = os.getenv('POSTGRES_PORT', '5432')
POSTGRES_DB = os.getenv('POSTGRES_DB')
POSTGRES_USER = os.getenv('POSTGRES_USER')
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')

S3_ENDPOINT = os.getenv('S3_ENDPOINT', 'rustfs:9000')
RUSTFS_USER = os.getenv('RUSTFS_USER', 'admin')
RUSTFS_PASSWORD = os.getenv('RUSTFS_PASSWORD')
RUSTFS_SSL = os.getenv('RUSTFS_SSL', 'false')
RUSTFS_BUCKET = os.getenv('RUSTFS_BUCKET', 'mitma')

print("✅ Variables de entorno cargadas")


✅ Variables de entorno cargadas


In [22]:
# Crear conexión a DuckDB
con = duckdb.connect()

# Instalar y cargar extensiones necesarias
critical_extensions = ['ducklake', 'postgres', 'httpfs']
for ext in critical_extensions:
    try:
        con.execute(f"INSTALL {ext};")
        con.execute(f"LOAD {ext};")
        print(f"✅ Extensión {ext} cargada")
    except Exception as e:
        print(f"⚠️ Advertencia al cargar {ext}: {e}")
        try:
            con.execute(f"LOAD {ext};")
            print(f"✅ Extensión {ext} cargada (ya estaba instalada)")
        except Exception as e2:
            print(f"❌ Error al cargar {ext}: {e2}")

# Cargar extensión spatial (opcional)
try:
    con.execute("INSTALL spatial;")
    con.execute("LOAD spatial;")
    print("✅ Extensión spatial cargada")
except Exception as e:
    print(f"⚠️ Spatial extension no disponible: {e}")


✅ Extensión ducklake cargada
✅ Extensión postgres cargada
✅ Extensión httpfs cargada
✅ Extensión spatial cargada


In [23]:
# Configurar S3 para RustFS
con.execute(f"SET s3_endpoint='{S3_ENDPOINT}';")
con.execute(f"SET s3_access_key_id='{RUSTFS_USER}';")
con.execute(f"SET s3_secret_access_key='{RUSTFS_PASSWORD}';")
con.execute(f"SET s3_use_ssl={RUSTFS_SSL};")
con.execute("SET s3_url_style='path';")

print("✅ Configuración de S3/RustFS completada")


✅ Configuración de S3/RustFS completada


In [24]:
# Attach DuckLake con PostgreSQL Catalog
postgres_connection_string = f"""
    dbname={POSTGRES_DB} host={POSTGRES_HOST} user={POSTGRES_USER} password={POSTGRES_PASSWORD} port={POSTGRES_PORT} 
    sslmode=require connect_timeout=30 keepalives=1 keepalives_idle=30 keepalives_interval=10 keepalives_count=5 tcp_user_timeout=30000
"""

# Verificar si ducklake ya está attachado
try:
    databases = con.execute("SELECT database_name FROM duckdb_databases();").fetchdf()
    if 'ducklake' not in databases['database_name'].values:
        attach_query = f"""
            ATTACH 'ducklake:postgres:{postgres_connection_string}' AS ducklake (DATA_PATH 's3://{RUSTFS_BUCKET}/');
        """
        con.execute(attach_query)
        print("✅ DuckLake attachado correctamente")
    else:
        print("✅ DuckLake ya está attachado")
except Exception as e:
    print(f"⚠️ Advertencia al attachar DuckLake: {e}")

# Usar la base de datos ducklake
con.execute("USE ducklake;")
print("✅ Usando base de datos ducklake")

✅ DuckLake attachado correctamente
✅ Usando base de datos ducklake


In [25]:
# Wrapper SQL para simplificar consultas y liberar memoria
import gc

class SQL:
    """Wrapper para ejecutar queries SQL de forma simple y liberar memoria"""
    def __init__(self, connection):
        self.con = connection
    
    def __call__(self, query):
        """Ejecuta una query y retorna el resultado como DataFrame (se muestra automáticamente en notebooks).
        Libera memoria de DuckDB después de ejecutar la query."""
        result = self.con.execute(query).fetchdf()
        # Liberar memoria de DuckDB después de fetchdf
        try:
            # Forzar liberación de memoria de DuckDB
            self.con.execute("PRAGMA force_checkpoint")
        except:
            pass

        gc.collect()
        return result
    
    def fetch_df(self, query):
        """Ejecuta una query y retorna un DataFrame"""
        result = self.con.execute(query).fetchdf()
        try:
            self.con.execute("PRAGMA force_checkpoint")
        except:
            pass
        return result

# Crear instancia global SQL
SQL = SQL(con)
print("✅ Wrapper SQL creado. Usa SQL(query) para ejecutar consultas - el DataFrame se mostrará automáticamente")
print("✅ El wrapper libera memoria automáticamente después de cada consulta")


✅ Wrapper SQL creado. Usa SQL(query) para ejecutar consultas - el DataFrame se mostrará automáticamente
✅ El wrapper libera memoria automáticamente después de cada consulta


In [26]:
SQL("SELECT * FROM bronze_mitma_od_municipios USING SAMPLE 10")

,fecha,periodo,origen,destino,distancia,actividad_origen,actividad_destino,estudio_origen_posible,estudio_destino_posible,residencia,renta,edad,sexo,viajes,viajes_km,loaded_at,source_file
0,20230425,13,08105,08135,2-10,trabajo_estudio,casa,no,no,08,10-15,45-65,NA,4.494,28.517,2026-01-01 21:13:50.400860+01:00,https://movilidad-opendata.mitma.es/estudios_b...
1,20230425,07,46244,46250,10-50,casa,trabajo_estudio,no,no,46,10-15,25-45,mujer,37.485,442.068,2026-01-01 21:13:50.400860+01:00,https://movilidad-opendata.mitma.es/estudios_b...
2,20230425,07,48002,48054,2-10,casa,trabajo_estudio,no,no,48,<10,NA,NA,9.287,67.668,2026-01-01 21:13:50.400860+01:00,https://movilidad-opendata.mitma.es/estudios_b...
3,20230425,07,50025,50025,0.5-2,casa,trabajo_estudio,no,no,50,10-15,NA,NA,11.169,13.841,2026-01-01 21:13:50.400860+01:00,https://movilidad-opendata.mitma.es/estudios_b...
4,20230425,14,12082,46128_AM,>50,trabajo_estudio,no_frecuente,no,no,46,10-15,45-65,mujer,2.568,334.241,2026-01-01 21:13:50.400860+01:00,https://movilidad-opendata.mitma.es/estudios_b...
5,20230425,07,48034,20032,2-10,casa,frecuente,no,no,48,10-15,65-100,NA,9.491,85.701,2026-01-01 21:13:50.400860+01:00,https://movilidad-opendata.mitma.es/estudios_b...
6,20230425,13,08101,08101,0.5-2,frecuente,casa,no,no,08,10-15,25-45,hombre,574.033,573.889,2026-01-01 21:13:50.400860+01:00,https://movilidad-opendata.mitma.es/estudios_b...
7,20230425,13,08125,08125,2-10,frecuente,frecuente,no,no,08,10-15,65-100,NA,2.49,8.374,2026-01-01 21:13:50.400860+01:00,https://movilidad-opendata.mitma.es/estudios_b...
8,20230425,13,08107,08019,10-50,casa,trabajo_estudio,no,no,08,10-15,45-65,NA,2.372,60.981,2026-01-01 21:13:50.400860+01:00,https://movilidad-opendata.mitma.es/estudios_b...
9,20230425,13,08019,08101,0.5-2,trabajo_estudio,frecuente,no,no,08,10-15,0-25,mujer,11.811,12.822,2026-01-01 21:13:50.400860+01:00,https://movilidad-opendata.mitma.es/estudios_b...


In [27]:
print("📊 Listando TODAS las tablas en la base de datos:")
SQL("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'main'
    ORDER BY table_name;
""")

📊 Listando TODAS las tablas en la base de datos:


,table_name
0,_temp_od_enriched
1,bronze_ine_empresas_municipio
2,bronze_ine_municipios
3,bronze_ine_poblacion_municipio
4,bronze_ine_renta_municipio
5,bronze_mitma_ine_relations
6,bronze_mitma_municipios
7,bronze_mitma_od_municipios
8,bronze_mitma_overnight_stay_municipios
9,bronze_mitma_people_day_municipios


In [28]:
# Filtrar tablas que comienzan con 'silver_' usando SQL wrapper
SQL("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'main'
        AND table_name LIKE 'silver_%'
    ORDER BY table_name;
""")

,table_name
0,silver_ine_all
1,silver_mitma_distances
2,silver_mitma_ine_mapping
3,silver_mitma_od
4,silver_mitma_od_processed_dates
5,silver_mitma_od_quality
6,silver_overnight_stay
7,silver_people_day
8,silver_zones


In [29]:
# silver_zones
SQL("SELECT * FROM silver_zones USING SAMPLE 10")

,id,nombre,geometry_obj,centroid
0,25228,Torrefarrera,"[3, 4, 0, 0, 0, 0, 0, 0, 232, 157, 14, 63, 37,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,05016,Arévalo,"[3, 4, 0, 0, 0, 0, 0, 0, 163, 7, 151, 192, 7, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,36052,Silleda,"[3, 4, 0, 0, 0, 0, 0, 0, 149, 108, 4, 193, 222...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,44158_AM,Mora de Rubielos agregacion de municipios,"[3, 4, 0, 0, 0, 0, 0, 0, 253, 23, 68, 191, 185...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,13033_AM,Castellar de Santiago agregacion de municipios,"[3, 4, 0, 0, 0, 0, 0, 0, 51, 254, 73, 192, 139...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
5,22901_AM,Valle de Hecho agregacion de municipios,"[3, 4, 0, 0, 0, 0, 0, 0, 92, 60, 56, 191, 227,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
6,26089,Logroño,"[3, 4, 0, 0, 0, 0, 0, 0, 98, 21, 29, 192, 244,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
7,33072_AM,Teverga agregacion de municipios,"[3, 4, 0, 0, 0, 0, 0, 0, 89, 156, 196, 192, 16...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
8,17164_AM,Sant Hilari Sacalm agregacion de municipios,"[3, 4, 0, 0, 0, 0, 0, 0, 37, 201, 28, 64, 232,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
9,32019,"Carballiño, O","[3, 4, 0, 0, 0, 0, 0, 0, 90, 60, 1, 193, 0, 19...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


In [30]:
SQL("SELECT * FROM silver_mitma_od USING SAMPLE 10")

,fecha,origen_zone_id,destino_zone_id,viajes,viajes_km,residencia
0,2023-03-06 06:00:00,20069,20073,153.234,925.470,20
1,2023-03-06 11:00:00,08019,08122_AM,14.259,579.923,08
2,2023-03-06 06:00:00,22225,22112_AM,2.964,118.656,22
3,2023-03-06 19:00:00,08086,08086,584.784,1001.318,08
4,2023-03-06 15:00:00,17223_AM,17221,5.375,107.985,17
5,2023-03-06 12:00:00,25193_AM,25051_AM,9.652,120.165,25
6,2023-03-06 00:00:00,39102,39079,2.814,21.720,39
7,2023-03-06 22:00:00,14026_AM,14052,38.238,716.390,14
8,2023-03-06 23:00:00,28082,28079,6.500,192.195,28
9,2023-03-06 01:00:00,40194,40194,37.365,58.255,28


In [43]:
# silver_od
SQL("""
    SELECT 
        od.*,
        CASE WHEN h.date IS NOT NULL THEN TRUE ELSE FALSE END AS is_holiday,
    FROM silver_mitma_od od
    LEFT JOIN bronze_spanish_holidays h 
        ON DATE(od.fecha) = h.date
    WHERE fecha >= '2023-04-07' 
    AND fecha < '2023-04-08'
    LIMIT 10;
""")

,fecha,origen_zone_id,destino_zone_id,viajes,viajes_km,residencia,is_holiday
0,2023-04-07 16:00:00,25040,25062_AM,114.383,1428.384,25,True
1,2023-04-07 16:00:00,25120,25048_AM,121.731,1664.425,25,True
2,2023-04-07 16:00:00,25173_AM,25016_AM,2.427,215.979,25,True
3,2023-04-07 16:00:00,25223_AM,08011_AM,6.131,146.583,08,True
4,2023-04-07 16:00:00,25228,22053_AM,4.395,103.949,08,True
5,2023-04-07 16:00:00,25240_AM,25040,172.479,617.844,25,True
6,2023-04-07 16:00:00,26089,31191_AM,5.109,397.288,18,True
7,2023-04-07 16:00:00,26089,31251_AM,2.061,16.102,09,True
8,2023-04-07 16:00:00,27005,27066,4.333,139.948,32,True
9,2023-04-07 16:00:00,27016,27058_AM,163.514,1534.759,27,True


In [38]:
SQL("""
    SELECT * 
    FROM silver_mitma_od 
    WHERE fecha >= '2023-03-06 00:00:00'::TIMESTAMP 
    AND fecha < '2023-03-08 00:00:00'::TIMESTAMP
    LIMIT 10;
""")

,fecha,origen_zone_id,destino_zone_id,viajes,viajes_km,residencia
0,2023-03-06 04:00:00,28149,28079,19.644,493.431000,28
1,2023-03-06 04:00:00,29008,29038,31.840,340.690000,29
2,2023-03-06 04:00:00,29067,29008,31.081,628.693000,29
3,2023-03-06 04:00:00,29901,29018_AM,2.657,121.667000,48
4,2023-03-06 04:00:00,30041,30041,175.678,337.453000,30
5,2023-03-06 04:00:00,31076_AM,31201,155.970,753.306482,31
6,2023-03-06 04:00:00,31201,31906_AM,124.791,414.506000,31
7,2023-03-06 04:00:00,32076_AM,32081_AM,12.576,262.646000,32
8,2023-03-06 04:00:00,33004,33004,555.864,1094.642000,33
9,2023-03-06 04:00:00,33014,33014,57.509,140.681000,33


In [45]:
SQL("""
ALTER TABLE silver_mitma_od ADD COLUMN is_weekend BOOLEAN;
ALTER TABLE silver_mitma_od ADD COLUMN is_holiday BOOLEAN;
""")

,Success


In [53]:
SQL("""
SET preserve_insertion_order=false;
SET enable_object_cache=true;
SET threads=6;
SET memory_limit='10GB';
UPDATE silver_mitma_od od
SET 
    is_weekend = EXTRACT(DOW FROM DATE(od.fecha)) IN (0, 6),
    is_holiday = CASE WHEN h.date IS NOT NULL THEN TRUE ELSE FALSE END
FROM bronze_spanish_holidays h
WHERE DATE(od.fecha) = h.date
  AND EXTRACT(MONTH FROM DATE(od.fecha)) = 4
  AND EXTRACT(YEAR FROM DATE(od.fecha)) = 2023;
""")

RuntimeError: Query interrupted

In [ ]:
SQL("SELECT * FROM bronze_spanish_holidays USING SAMPLE 10")

,date,name
0,2023-01-06,Epifanía del Señor
1,2023-04-07,Viernes Santo
2,2023-05-01,Día del Trabajador
3,2023-08-15,Asunción de la Virgen
4,2023-10-12,Fiesta Nacional de España
5,2023-11-01,Todos los Santos
6,2023-12-06,Día de la Constitución Española
7,2023-12-08,Inmaculada Concepción
8,2023-12-25,Navidad


In [28]:
SQL("""
    SELECT COUNT(*) FROM silver_mitma_od
    UNION ALL
    SELECT COUNT(*) FROM bronze_mitma_od_municipios
    UNION ALL
    SELECT COUNT(*) FROM silver_mitma_od_quality
""")

,count_star()
0,140825079
1,741162633
2,156340230


In [29]:
# silver_od_quality
SQL("SELECT * FROM silver_mitma_od_quality USING SAMPLE 10")


,fecha,origen_zone_id,destino_zone_id,residencia,viajes_per_capita,km_per_capita,flag_negative_viajes,flag_negative_viajes_km,z_viajes_per_capita,flag_outlier_viajes_per_capita
0,2023-04-11 07:00:00,29067,29094,29,0.001219,0.033488,False,False,-0.154245,False
1,2023-04-11 08:00:00,13034,41091,13,0.000031,0.007851,False,False,-0.292059,False
2,2023-04-11 11:00:00,39025,39060,39,0.003793,0.040828,False,False,0.144284,False
3,2023-04-11 21:00:00,25120,22225,25,0.000036,0.001049,False,False,-0.291550,False
4,2023-04-11 17:00:00,46207,46013,46,0.003418,0.034503,False,False,0.100818,False
5,2023-04-11 10:00:00,28059,28005,28,0.001006,0.018243,False,False,-0.178938,False
6,2023-04-11 16:00:00,48060_AM,48095_AM,48,0.003189,0.023270,False,False,0.074318,False
7,2023-04-11 16:00:00,11014,11007,29,0.000538,0.008891,False,False,-0.233316,False
8,2023-04-11 07:00:00,47129_AM,47010,47,0.000835,0.028870,False,False,-0.198837,False
9,2023-04-11 17:00:00,03092_AM,46200_AM,03,0.000185,0.001366,False,False,-0.274172,False


In [ ]:
# silver_overnight_stay
SQL("SELECT * FROM silver_overnight_stay USING SAMPLE 10")


,fecha,zona_pernoctacion,zona_residencia,personas
0,2023-03-02,28079,2702804_AD,53.827
1,2023-03-02,28145,28009,3.039
2,2023-03-02,41038,28014,14.980
3,2023-03-01,27005,27024_AM,7.306
4,2023-03-02,34056_AM,0105902,3.170
5,2023-03-02,16078,0200303,7.500
6,2023-03-02,49178_AM,4718610,12.549
7,2023-03-02,39082_AM,4802006,4.137
8,2023-03-01,27021_AM,1503006,9.185
9,2023-03-01,02003,23082_AM,22.500


In [ ]:
# silver_people_day
SQL("SELECT * FROM silver_people_day USING SAMPLE 10")


,fecha,zona_pernoctacion,edad,sexo,numero_viajes,personas
0,2023-03-02,06159,25-45,hombre,2+,155.203
1,2023-03-02,08136,65-100,mujer,1,48.698
2,2023-03-02,08282,0-25,hombre,1,27.397
3,2023-03-02,11015,65-100,mujer,1,184.052
4,2023-03-02,12031,65-100,mujer,1,12.456
5,2023-03-02,15009,65-100,hombre,2,302.083
6,2023-03-01,03064,65-100,hombre,0,264.871
7,2023-03-01,28168_AM,0-25,hombre,2,64.415
8,2023-03-01,22907_AM,65-100,hombre,2,130.622
9,2023-03-01,25204_AM,45-65,mujer,2,171.379


In [ ]:
# silver_mitma_distances
SQL("SELECT * FROM silver_mitma_distances USING SAMPLE 10")


,origin,destination,distance_km
0,02045,02073_AM,15.915524
1,02045,04029,293.576838
2,03043,05002_AM,475.104811
3,04064,05168_AM,473.471941
4,01051,14049,657.831267
5,03053,17164_AM,505.905016
6,12902_AM,17169_AM,378.949676
7,08097_AM,20022_AM,511.946709
8,08088,23050,797.580336
9,08125,23055,749.165117


In [ ]:
# silver_mitma_ine_mapping
SQL("SELECT * FROM silver_mitma_ine_mapping USING SAMPLE 10")


,nombre,codigo_ine,municipio_mitma
0,barbadillo de herreros,09037,09330_AM
1,bohonal de ibor,10030,10140_AM
2,fuentes de ayodar,12064,12072_AM
3,algar de mesa,19016,19086_AM
4,zufre,21079,21069_AM
5,garganta de los montes,28062,28027_AM
6,barillas,31048,31068_AM
7,malpartida,37178,37174_AM
8,"riba de escalote, la",42157,42035_AM
9,canizal,49034,49081_AM


In [ ]:
# silver_ine_all
SQL("SELECT * FROM silver_ine_all USING SAMPLE 10")


,id,nombre,empresas,renta_media,poblacion_total,poblacion_hombres,poblacion_mujeres,year
0,08068,Cervelló,803.0,18664.0,9743.0,4911.0,4832.0,
1,10116_AM,Malpartida de Plasencia agregacion de municipios,319.0,11008.0,5236.0,2608.0,2628.0,
2,13092_AM,Villanueva de la Fuente agregacion de municipios,402.0,11341.0,6142.0,3198.0,2944.0,
3,19143,Horche,126.0,13502.0,3119.0,1596.0,1523.0,
4,22907_AM,Aínsa-Sobrarbe agregacion de municipios,424.0,17618.0,5222.0,2712.0,2510.0,
5,28137,"Santos de la Humosa, Los",126.0,13849.0,2867.0,1486.0,1381.0,
6,37156,Guijuelo,525.0,13971.0,5427.0,2776.0,2651.0,
7,40906_AM,San Cristóbal de Segovia agregacion de municipios,388.0,14781.0,5796.0,2955.0,2841.0,
8,50099,Épila,219.0,12570.0,4704.0,2431.0,2273.0,
9,50219,"Puebla de Alfindén, La",708.0,14581.0,6550.0,3356.0,3194.0,


In [32]:
# silver_zones_all
SQL("SELECT * FROM silver_zones_all USING SAMPLE 10")


,id,nombre,geometry_obj,centroid
0,07001,Alaró,"[3, 4, 0, 0, 0, 0, 0, 0, 125, 90, 51, 64, 230,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,07033,Manacor,"[3, 4, 0, 0, 0, 0, 0, 0, 119, 17, 79, 64, 34, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,20036,Hondarribia,"[3, 4, 0, 0, 0, 0, 0, 0, 129, 180, 233, 191, 4...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,29090_AM,Tolox agregacion de municipios,"[3, 4, 0, 0, 0, 0, 0, 0, 55, 113, 157, 192, 14...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,08149_AM,Olost agregacion de municipios,"[3, 4, 0, 0, 0, 0, 0, 0, 194, 162, 6, 64, 131,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
5,36038,Pontevedra,"[3, 4, 0, 0, 0, 0, 0, 0, 235, 234, 9, 193, 170...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
6,45118_AM,Novés agregacion de municipios,"[3, 4, 0, 0, 0, 0, 0, 0, 199, 76, 136, 192, 16...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
7,15024,Cerceda,"[3, 4, 0, 0, 0, 0, 0, 0, 16, 189, 7, 193, 42, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
8,47165,Tordesillas,"[3, 4, 0, 0, 0, 0, 0, 0, 216, 47, 160, 192, 24...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
9,12027,Benicarló,"[3, 4, 0, 0, 0, 0, 0, 0, 170, 67, 206, 62, 111...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
